In [1]:
import pandas as pd
import numpy as np

### Data visualization

In [2]:
data = pd.read_csv('Task 3 and 4_Loan_Data.csv')

In [3]:
data

,customer_id,credit_lines_outstanding,loan_amt_outstanding,total_debt_outstanding,income,years_employed,fico_score,default
0,8153374,0,5221.545193,3915.471226,78039.38546,5,605,0
1,7442532,5,1958.928726,8228.752520,26648.43525,2,572,1
2,2256073,0,3363.009259,2027.830850,65866.71246,4,602,0
3,4885975,0,4766.648001,2501.730397,74356.88347,5,612,0
4,4700614,1,1345.827718,1768.826187,23448.32631,6,631,0
...,...,...,...,...,...,...,...,...
9995,3972488,0,3033.647103,2553.733144,42691.62787,5,697,0
9996,6184073,1,4146.239304,5458.163525,79969.50521,8,615,0
9997,6694516,2,3088.223727,4813.090925,38192.67591,5,596,0
9998,3942961,0,3288.901666,1043.099660,50929.37206,2,647,0


### Log Likelihood

In [4]:
# fico = data["fico_score"].dropna()
# default = data["default"]

In [5]:
# def log_likelihood_split(scores, defaults, n_buckets):
#     """
#     Split range of FICO scores en n_buckets en maximisant la log-likelihood.
#     """
#     scores_sorted = scores.sort_values().reset_index(drop=True)
#     defaults_sorted = defaults.loc[scores_sorted.index].reset_index(drop=True)

#     N = len(scores_sorted)
#     dp = np.full((n_buckets + 1, N + 1), -np.inf)
#     split = np.zeros((n_buckets + 1, N + 1), dtype=int)
#     dp[0, 0] = 0  # base case

#     # Dynamic programming
#     for k in range(1, n_buckets + 1):
#         for j in range(1, N + 1):
#             for i in range(k - 1, j):
#                 ni = j - i
#                 ki = defaults_sorted[i:j].sum()
#                 pi = ki / ni if ni > 0 else 1e-6
#                 if pi in [0, 1]:
#                     ll = 0  # éviter log(0)
#                 else:
#                     ll = ki * np.log(pi) + (ni - ki) * np.log(1 - pi)

#                 if dp[k-1, i] + ll > dp[k, j]:
#                     dp[k, j] = dp[k-1, i] + ll
#                     split[k, j] = i

#     # Récupérer les bornes
#     boundaries = []
#     j = N
#     for k in range(n_buckets, 0, -1):
#         i = split[k, j]
#         boundaries.append((scores_sorted[i], scores_sorted[j-1]))
#         j = i

#     return sorted(boundaries, key=lambda x: x[0])

# # Appliquer
# ll_map = log_likelihood_split(fico, default, n_buckets=5)
# print("\n--- Rating map (Log-likelihood) ---")
# for idx, (low, high) in enumerate(ll_map, 1):
#     print(f"Rating {idx}: {low:.0f} - {high:.0f}")

In [11]:
def calc_log_likelihood(data, boundaries, eps=1e-8):
    """Calcule la log-vraisemblance pour un découpage donné des FICO scores."""
    scores = data['fico_score'].values
    defaults = data['default'].values
    bins = np.digitize(scores, boundaries, right=False) - 1
    num_buckets = len(boundaries) - 1

    # Forcer la taille de bincount pour correspondre exactement à num_buckets
    ni = np.bincount(bins, minlength=num_buckets)[:num_buckets]
    ki = np.bincount(bins, weights=defaults, minlength=num_buckets)[:num_buckets]

    # Lissage Laplace
    pi = (ki + eps) / (ni + 2 * eps)

    ll_parts = np.zeros(num_buckets)
    mask = ni > 0
    ll_parts[mask] = ki[mask] * np.log(pi[mask]) + (ni[mask] - ki[mask]) * np.log(1 - pi[mask])
    return ll_parts.sum()

def comp_bic(log_likelihood, num_params, num_obs):
    """Critère BIC = -2*LL + p*log(N)."""
    return -2.0 * log_likelihood + num_params * np.log(num_obs)
    

In [12]:
def compute_cumulative_sums(values):
    """Prépare les sommes cumulées nécessaires au calcul O(1) des MSE."""
    n = len(values)
    S = np.zeros(n + 1)
    S2 = np.zeros(n + 1)
    for i in range(n):
        S[i+1] = S[i] + values[i]
        S2[i+1] = S2[i] + values[i] ** 2
    return S, S2

def mse(S, S2, i, j):
    """Erreur quadratique pour le segment [i:j)."""
    n = j - i
    if n == 0:
        return 0
    total = S[j] - S[i]
    total2 = S2[j] - S2[i]
    return total2 - (total ** 2) / n


In [13]:
def opt_buckets(data, num_buckets):
    """Retourne les bornes optimales (par DP) minimisant la MSE."""
    values = np.sort(data['fico_score'].values)
    n = len(values)
    S, S2 = compute_cumulative_sums(values)

    dp = np.full((num_buckets + 1, n + 1), np.inf)
    parent = np.zeros((num_buckets + 1, n + 1), dtype=int)
    dp[0,0] = 0

    for b in range(1, num_buckets + 1):
        for j in range(1, n + 1):
            for i in range(b-1, j):
                cost = dp[b-1,i] + mse(S, S2, i, j)
                if cost < dp[b,j]:
                    dp[b,j] = cost
                    parent[b,j] = i

    # Reconstruction des bornes
    boundaries = [n]
    cur = n
    for b in range(num_buckets, 0, -1):
        cur = parent[b,cur]
        boundaries.append(cur)
    boundaries = sorted(boundaries)

    # Conversion indices -> valeurs
    result = [values[0]]
    for idx in boundaries[1:-1]:
        result.append(values[idx])
    result.append(values[-1])
    return np.array(result)


In [14]:
def opt_bucket_count_bic(data, min_buckets=2, max_buckets=10):
    N = len(data)
    best_bic = np.inf
    best_count, best_boundaries, best_ll = None, None, None
    bic_scores = {}

    for k in range(min_buckets, max_buckets+1):
        boundaries = opt_buckets(data, k)
        ll = calc_log_likelihood(data, boundaries)
        bic = comp_bic(ll, k, N)
        bic_scores[k] = bic

        if bic < best_bic:
            best_bic = bic
            best_count = k
            best_boundaries = boundaries
            best_ll = ll

    return best_count, best_boundaries, bic_scores, best_ll


In [15]:
%%time
best_count, best_boundaries, bic_scores, best_ll = opt_bucket_count_bic(data, 2, 10)

CPU times: user 1h 1min 9s, sys: 3.17 s, total: 1h 1min 12s
Wall time: 1h 1min 14s


In [16]:
%%time
print(f"Best number of buckets {best_count}, with log-likelihood: {best_ll}")

Best number of buckets 10, with log-likelihood: -4239.954262034134
CPU times: user 92 μs, sys: 0 ns, total: 92 μs
Wall time: 81.5 μs


In [17]:
%%time
# Assigner les buckets optimaux
data['bucket'] = pd.cut(data['fico_score'], bins=best_boundaries,
                           labels=False, include_lowest=True)
data.head()

CPU times: user 1.47 ms, sys: 4 ms, total: 5.47 ms
Wall time: 4.25 ms


,customer_id,credit_lines_outstanding,loan_amt_outstanding,total_debt_outstanding,income,years_employed,fico_score,default,bucket
0,8153374,0,5221.545193,3915.471226,78039.38546,5,605,0,3
1,7442532,5,1958.928726,8228.752520,26648.43525,2,572,1,2
2,2256073,0,3363.009259,2027.830850,65866.71246,4,602,0,3
3,4885975,0,4766.648001,2501.730397,74356.88347,5,612,0,4
4,4700614,1,1345.827718,1768.826187,23448.32631,6,631,0,4
